## Goals:
- Data Uniqueness Check
- Data Completeness Check
- Data Validity Check
- Data Consistency Check
- Data Timeliness Check
- Data Preparation

In [17]:
import pandas as pd
import sqlite3

In [23]:
# Create a connection to the SQLite database
conn=sqlite3.connect("data/bootcamp_db")

In [33]:
# Check the connection
conn

In [35]:
# Write a query to read all tables from the database
sql_query = """SELECT name 
               FROM sqlite_master 
               WHERE type='table';"""

cursor = conn.cursor()

# Executing our sql query
cursor.execute(sql_query)

# Printing all tables list
print(f"List of tables in the database:\n{cursor.fetchall()}")

List of tables in the database:
[('online_transactions',), ('stock_description',)]


In [37]:
# Write a query that reads all the rows from online transactions table 
query="""
select *
from online_transactions
"""

# Read from the database into a Pandas DataFrame
online_transactions=pd.read_sql(query,conn)

In [38]:
# Check the result
online_transactions.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
0,536370,21791,24,2010-12-01 08:45:00,1.25,u12583,France
1,536373,82494L,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom
2,536378,21929,10,2010-12-01 09:37:00,1.95,u14688,United Kingdom
3,536381,37444A,1,2010-12-01 09:41:00,2.95,u15311,United Kingdom
4,536381,15056BL,2,2010-12-01 09:41:00,5.95,u15311,United Kingdom


In [41]:
# Write a query that reads all the rows from the stock description table 
query="""
select *
from stock_description
"""

# Read from the database into a Pandas DataFrame
stock_description=pd.read_sql(query,conn)

In [43]:
# Check the result
stock_description.head()

,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


## 1. Data Uniqueness Check(Duplicates)

In [45]:
# Identify duplicated rows of data
online_transactions[online_transactions.duplicated(keep=False)]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
30,536412,22273,1,2010-12-01 11:49:00,2.95,u1792,United Kingdom
70,536528,22744,2,2010-12-01 13:17:00,2.95,u15525,United Kingdom
111,536412,21706,1,2010-12-01 11:49:00,4.95,u1792,United Kingdom
133,536522,21121,1,2010-12-01 12:49:00,1.25,u15012,United Kingdom
139,536528,22839,1,2010-12-01 13:17:00,14.95,u15525,United Kingdom
...,...,...,...,...,...,...,...
541644,581352,82494L,1,2011-12-08 12:26:00,2.95,u14698,United Kingdom
541669,581404,22469,2,2011-12-08 13:47:00,1.65,u1368,United Kingdom
541888,581538,22068,1,2011-12-09 11:34:00,0.39,u14446,United Kingdom
541890,581538,23275,1,2011-12-09 11:34:00,1.25,u14446,United Kingdom


In [47]:
# Number of duplicated rows -- this count does not include the first occurance of the row of data
online_transactions.duplicated().sum()

5270

In [49]:
# An example of duplicated data
# Both rows are identical
online_transactions[(online_transactions.invoice=="536412") & (online_transactions.stock_code=="22273")]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
30,536412,22273,1,2010-12-01 11:49:00,2.95,u1792,United Kingdom
33,536412,22273,2,2010-12-01 11:49:00,2.95,u1792,United Kingdom
9624,536412,22273,1,2010-12-01 11:49:00,2.95,u1792,United Kingdom


In [57]:
# Check the duplicates in stock_description DataFrame
stock_description[stock_description.duplicated()].shape[0]

0

## 2. Data Completeness Check（Missing Values）

### 2.1 Null values

In [60]:
# Check for missing values in online transactions
online_transactions.isnull().sum()

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
dtype: int64

In [62]:
# Check for missing values in stock description
stock_description.isnull().sum()

stock_code     0
description    0
dtype: int64

### 2.2 Empty string

In [73]:
# Find all empty strings from online_transactions 
empty_string_ot=(online_transactions=='')

In [75]:
# Count empty strings in each column
empty_string_ot_count=empty_string_ot.sum()

In [85]:
# Check the result
empty_string_ot_count

invoice              0
stock_code           0
quantity             0
invoice_date         0
price                0
customer_id     135080
country              0
dtype: int64

In [87]:
# Show the empty rows from the DataFrame
online_transactions[online_transactions.customer_id==""]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
157,536544,21809,1,2010-12-01 14:32:00,2.51,,United Kingdom
158,536544,21991,1,2010-12-01 14:32:00,2.51,,United Kingdom
159,536544,22076,1,2010-12-01 14:32:00,3.36,,United Kingdom
160,536544,22135,5,2010-12-01 14:32:00,0.85,,United Kingdom
161,536544,22190,3,2010-12-01 14:32:00,2.51,,United Kingdom
...,...,...,...,...,...,...,...
541876,581498,22557,1,2011-12-09 10:26:00,3.29,,United Kingdom
541877,581498,22993,2,2011-12-09 10:26:00,3.29,,United Kingdom
541878,581498,23300,9,2011-12-09 10:26:00,3.29,,United Kingdom
541879,581498,23480,1,2011-12-09 10:26:00,7.46,,United Kingdom


In [89]:
# Find all empty strings from online_transactions 
empty_string_sd=(stock_description=='')

In [91]:
# Count empty strings in each column
empty_string_sd_count=empty_string_sd.sum()

In [93]:
# Check the result
empty_string_sd_count

stock_code     0
description    0
dtype: int64

## 3. Data Validity Check

In [29]:
# Check the summary of the dataframes? There should be no negative values
online_transactions.describe()

,quantity,price
count,541910.000000,541910.000000
mean,9.552234,4.611138
std,218.080957,96.759765
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [98]:
# Find transactions with negative prices
online_transactions[online_transactions.price<0]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
197892,A563187,B,1,2011-08-12 14:52:00,-11062.06,,United Kingdom
205990,A563186,B,1,2011-08-12 14:51:00,-11062.06,,United Kingdom


In [105]:
# Find transactions with zero prices
online_transactions[online_transactions.price==0]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
383,536546,22145,1,2010-12-01 14:33:00,0.0,,United Kingdom
386,536554,84670,23,2010-12-01 14:35:00,0.0,,United Kingdom
779,536765,84952C,19,2010-12-02 14:43:00,0.0,,United Kingdom
1555,537534,22501,2,2010-12-07 11:48:00,0.0,,United Kingdom
1556,537534,20652,1,2010-12-07 11:48:00,0.0,,United Kingdom
...,...,...,...,...,...,...,...
538379,579683,21868,-28,2011-11-30 13:35:00,0.0,,United Kingdom
538608,579733,84581,-41,2011-11-30 14:24:00,0.0,,United Kingdom
538629,579755,84912B,-14,2011-11-30 14:54:00,0.0,,United Kingdom
538967,580366,22923,6,2011-12-02 16:38:00,0.0,,United Kingdom


## 4. Data Consistency Check

In [114]:
# Check stock code data types
print("Data type of 'stock_code' in online_transactions:", online_transactions['stock_code'].dtype)
print("Data type of 'stock_code' in stock_description:", stock_description['stock_code'].dtype)

Data type of 'stock_code' in online_transactions: object
Data type of 'stock_code' in stock_description: object


In [125]:
# Define the expected format for purely alphabetic text
pure_text_format = re.compile(r'^[A-Za-z ]+$')

In [127]:
# Identify stock codes that are purely alphabetic text
pure_text_stock_codes = online_transactions[online_transactions['stock_code'].apply(lambda x: bool(pure_text_format.match(x)))]

In [129]:
# Remove duplicates
unique_pure_text_stock_codes = pure_text_stock_codes['stock_code'].unique()

In [133]:
print("Unique purely alphabetic stock codes:")
print(unique_pure_text_stock_codes)

Unique purely alphabetic stock codes:
['BANK CHARGES' 'DOT' 'POST' 'D' 'AMAZONFEE' 'M' 'S' 'm' 'DCGSSBOY'
 'DCGSSGIRL' 'PADS' 'B' 'CRUK']


In [137]:
# Display summary statistics of the DataFrame
stock_description.describe()

,stock_code,description
count,3952,3952
unique,3905,3785
top,22600,?
freq,2,47


In [139]:
# Explore the stock_code field - do both tables contain the same number of stock_codes?
online_transactions['stock_code'].shape[0]

541910

In [141]:
stock_description['stock_code'].shape[0]

3952

In [143]:
# It is ok if the stock_description table has more stock codes as it is possible some items have not been purchased yet, 
# But what if a customer has purchased an item and we have no information about the stock code? 
# Is that a data quality issue?

In [145]:
len(stock_description['description'])

3952

In [149]:
# The number of dultiple descriptions from stock stock
stock_description[stock_description['stock_code'].duplicated()].shape[0]

47

In [151]:
# The number of stock codes have '?' in their description
stock_description['description'].isin(['?']).sum()

47

## 5. Data Timeliness Check

In [171]:
# Check for missing values
online_transactions['invoice_date'].isnull().sum()

0

In [184]:
# Check the start of dates
online_transactions['invoice_date'].min()

'2010-12-01 08:26:00'

In [186]:
# Check the end of dates 
online_transactions['invoice_date'].max()

'2011-12-09 12:50:00'

In [188]:
# Check for frequency of updates
online_transactions['invoice_date'].value_counts()

invoice_date
2011-10-31 14:41:00    1114
2011-12-08 09:28:00     749
2011-12-09 10:03:00     731
2011-12-05 17:24:00     721
2011-06-29 15:58:00     705
                       ... 
2011-07-01 15:07:00       1
2011-07-15 09:51:00       1
2011-07-15 09:53:00       1
2011-02-02 11:55:00       1
2011-12-08 13:06:00       1
Name: count, Length: 23260, dtype: int64

## 6. Data Preparation

In [196]:
# SQL query for left join with data clean requirements
query="""
select ot.*,
       sd.description
from online_transactions ot
left join (select *
           from stock_description 
           where description <> '?') sd
on ot.stock_code=sd.stock_code
where customer_id <> ''
and price>0
and ot.stock_code not in ('BANK CHARGES','POST','D','M','CRUK')
"""
# Read the query into a DataFrame
ot_w_desc=pd.read_sql(query,conn)

In [197]:
# Show the number of rows and columns
ot_w_desc.shape

(405029, 8)

In [200]:
# Show the result
ot_w_desc.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536370,21791,24,2010-12-01 08:45:00,1.25,u12583,France,VINTAGE HEADS AND TAILS CARD GAME
1,536373,82494L,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,WOODEN FRAME ANTIQUE WHITE
2,536378,21929,10,2010-12-01 09:37:00,1.95,u14688,United Kingdom,JUMBO BAG PINK VINTAGE PAISLEY
3,536381,37444A,1,2010-12-01 09:41:00,2.95,u15311,United Kingdom,YELLOW BREAKFAST CUP AND SAUCER
4,536381,15056BL,2,2010-12-01 09:41:00,5.95,u15311,United Kingdom,EDWARDIAN PARASOL BLACK


In [202]:
# Check the null values
ot_w_desc.isnull().sum()

invoice            0
stock_code         0
quantity           0
invoice_date       0
price              0
customer_id        0
country            0
description     1175
dtype: int64

In [204]:
# Replace missing values in the 'description' column with 'UNKNOWN'
ot_w_desc['description'].fillna('UNKNOWN',inplace=True)

In [206]:
# Check the result
ot_w_desc.isnull().sum()

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
description     0
dtype: int64

In [208]:
# Get the counts of unique values in the 'description' column and display the top 20 most frequent values
ot_w_desc.description.value_counts().nlargest(20)

description
CREAM HANGING HEART T-LIGHT HOLDER    2077
REGENCY CAKESTAND 3 TIER              1904
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1415
LUNCH BAG RED RETROSPOT               1359
SET OF 3 CAKE TINS PANTRY DESIGN      1232
UNKNOWN                               1175
LUNCH BAG  BLACK SKULL.               1126
POPCORN HOLDER                        1118
JUMBO BAG VINTAGE DOILEY              1115
LUNCH BAG SUKI DESIGN                 1103
PACK OF 72 RETROSPOT CAKE CASES       1080
LUNCH BAG VINTAGE DOILEY              1040
BUNTING , SPOTTY                      1036
PAPER CHAIN KIT 50'S CHRISTMAS        1029
LUNCH BAG SPACEBOY DESIGN             1021
LUNCH BAG CARS BLUE                   1012
NATURAL SLATE HEART CHALKBOARD         997
HEART OF WICKER SMALL                  996
Name: count, dtype: int64

In [210]:
# Drop the duplicate rows of data
ot_cleaned=ot_w_desc.drop_duplicates(keep='first')

In [211]:
# Check the result 
ot_cleaned.shape

(399807, 8)

In [217]:
# Store as a .csv file to local data folder
ot_cleaned.to_csv('data/ot_final.csv',index=False)

In [222]:
# Read the .csv file
check=pd.read_csv('data/ot_final.csv')

In [227]:
# Check the result
check.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536370,21791,24,2010-12-01 08:45:00,1.25,u12583,France,VINTAGE HEADS AND TAILS CARD GAME
1,536373,82494L,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,WOODEN FRAME ANTIQUE WHITE
2,536378,21929,10,2010-12-01 09:37:00,1.95,u14688,United Kingdom,JUMBO BAG PINK VINTAGE PAISLEY
3,536381,37444A,1,2010-12-01 09:41:00,2.95,u15311,United Kingdom,YELLOW BREAKFAST CUP AND SAUCER
4,536381,15056BL,2,2010-12-01 09:41:00,5.95,u15311,United Kingdom,EDWARDIAN PARASOL BLACK
